## Intsalling requirements and creating spark session

In [ ]:
!pip install pyspark

In [454]:
!pip install haversine

Defaulting to user installation because normal site-packages is not writeable


In [393]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode", "client")
sc = SparkContext.getOrCreate(cf)

In [394]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("temp").config("spark.some.config.option", "some-value").getOrCreate()

In [395]:
cf

In [396]:
sc

<SparkContext master=yarn appName=pyspark-shell>

# Question 1

In [5]:
df1 = spark.read.csv('BreadBasket_DMS.csv', header=True)

In [6]:
df1.show()

+----------+--------+-----------+-------------+
|      Date|    Time|Transaction|         Item|
+----------+--------+-----------+-------------+
|2016-10-30|09:58:11|          1|        Bread|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:07:57|          3|Hot chocolate|
|2016-10-30|10:07:57|          3|          Jam|
|2016-10-30|10:07:57|          3|      Cookies|
|2016-10-30|10:08:41|          4|       Muffin|
|2016-10-30|10:13:03|          5|       Coffee|
|2016-10-30|10:13:03|          5|       Pastry|
|2016-10-30|10:13:03|          5|        Bread|
|2016-10-30|10:16:55|          6|    Medialuna|
|2016-10-30|10:16:55|          6|       Pastry|
|2016-10-30|10:16:55|          6|       Muffin|
|2016-10-30|10:19:12|          7|    Medialuna|
|2016-10-30|10:19:12|          7|       Pastry|
|2016-10-30|10:19:12|          7|       Coffee|
|2016-10-30|10:19:12|          7|          Tea|
|2016-10-30|10:20:51|          8|       

In [7]:
# time greater than equal to 9AM and less than 11PM
from datetime import datetime, timedelta
from dateutil.parser import parse
time_var_1 = parse('9:00 AM')
time_var_2 = parse('11:00 PM')
df1 = df1.filter((df1.Time >= time_var_1) & (df1.Time < time_var_2))
df1.createOrReplaceTempView("items")

In [8]:
df3 = spark.sql("SELECT Count(Item), Item FROM items GROUP BY Item ORDER BY Count(Item) DESC")

### Answer to Question 1

In [9]:
df3.show(5) #top 5 items

+-----------+------+
|count(Item)|  Item|
+-----------+------+
|       5259|Coffee|
|       3151| Bread|
|       1414|   Tea|
|       1017|  Cake|
|        797|Pastry|
+-----------+------+
only showing top 5 rows



# Question 2

In [141]:
rdf = df1 = spark.read.options(delimiter=';').csv('Restaurants_in_Durham_County_NC.csv', header=True)

In [142]:
rdf

DataFrame[ID: string, Premise_Name: string, Premise_Address1: string, Premise_Address2: string, Premise_City: string, Premise_State: string, Premise_Zip: string, Premise_Phone: string, Hours_Of_Operation: string, Opening_Date: string, Closing_Date: string, Seats: string, Water: string, Sewage: string, Insp_Freq: string, Est_Group_Desc: string, Risk: string, Smoking_Allowed: string, Type_Description: string, Rpt_Area_Desc: string, Status: string, Transitional_Type_Desc: string, geolocation: string]

In [143]:
rdf.select("Rpt_Area_Desc").show(10)

+----------------+
|   Rpt_Area_Desc|
+----------------+
|    Food Service|
|    Food Service|
|    Food Service|
|    Food Service|
|Residential Care|
|Residential Care|
|        Day Care|
|  Swimming Pools|
|  Swimming Pools|
|  Swimming Pools|
+----------------+
only showing top 10 rows



In [144]:
rdf.createOrReplaceTempView("entities")
rdf2 = spark.sql("SELECT Count(ID), Rpt_Area_Desc FROM entities GROUP BY Rpt_Area_Desc")

### Answer to Question 2

In [145]:
rdf2.show()

+---------+--------------------+
|count(ID)|       Rpt_Area_Desc|
+---------+--------------------+
|        4|  Bed&Breakfast Home|
|        4|        Summer Camps|
|       30|        Institutions|
|        2|   Local Confinement|
|      147|         Mobile Food|
|       89|    School Buildings|
|       13|                null|
|      242|         Summer Food|
|      420|      Swimming Pools|
|      173|            Day Care|
|       32|Tattoo Establishm...|
|      154|    Residential Care|
|        2|   Bed&Breakfast Inn|
|        5|      Adult Day Care|
|       62|             Lodging|
|     1093|        Food Service|
+---------+--------------------+



## Question 3

In [64]:
pdf = spark.read.csv('populationbycountry19802010millions.csv', header=True)

In [65]:
pdf.show(5)

+-------------+---------+---------+---------+---------+---------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|          _c0|     1980|     1981|     1982|     1983|     1984|     1985|     1986|     1987|    1988|     1989|    1990|     1991|     1992|     1993|    1994|     1995|    1996|     1997|     1998|     1999|    2000|     2001|     2002|     2003|     2004|     2005|     2006|     2007|     2008|     2009|     2010|
+-------------+---------+---------+---------+---------+---------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|North America|320.27638|324.44694|32

In [66]:
from pyspark.sql.functions import col, round, max, min
df_percent_change = pdf.select("_c0", *[round((((col(year) - col(str(int(year)-1))) / col(str(int(year)-1))) * 100), 2).alias(year) for year in df.columns[2:]])

# Stacking the columns to get the year and percent change in separate rows
columns_expr = [f"`{year}`, {year}" for year in df_percent_change.columns[1:]]
columns_str = ", ".join(columns_expr)
df_percent_change = df_percent_change.selectExpr(f"stack(31, {columns_str}) as (PercentChange, Year)", "_c0 as Country")

# Getting the country with the maximum population change for each year
df_max_percent_change = df_percent_change.groupBy("Year").agg(max("PercentChange").alias("MaxPercentChange"))
df_max_percent_change = df_max_percent_change.join(df_percent_change, (df_max_percent_change.Year == df_percent_change.Year) & (df_max_percent_change.MaxPercentChange == col("PercentChange")), "inner").select("Country", df_max_percent_change.Year, df_max_percent_change.MaxPercentChange)

# Getting the country with the minimum population change for each year
df_min_percent_change = df_percent_change.groupBy("Year").agg(min("PercentChange").alias("MinPercentChange"))
df_min_percent_change = df_min_percent_change.join(df_percent_change, (df_min_percent_change.Year == df_percent_change.Year) & (df_min_percent_change.MinPercentChange == col("PercentChange")), "inner").select("Country", df_min_percent_change.Year, df_min_percent_change.MinPercentChange)

In [67]:
df_max_percent_change.orderBy('Year').show(31)

+--------------------+----+----------------+
|             Country|Year|MaxPercentChange|
+--------------------+----+----------------+
|      Western Sahara|1981|           12.13|
|      Western Sahara|1982|           11.12|
|       French Guiana|1983|           14.29|
|        Saint Helena|1984|           10.96|
|               Qatar|1984|           10.96|
|       French Guiana|1985|            12.5|
|               Qatar|1986|            8.77|
|       French Guiana|1987|           11.11|
|      Cayman Islands|1988|           11.01|
|United Arab Emirates|1989|            6.12|
|            Djibouti|1990|           12.82|
|              Jordan|1991|           11.27|
|              Kuwait|1992|           48.63|
|         Afghanistan|1993|           13.22|
|         Afghanistan|1994|            8.73|
|             Burundi|1995|            7.22|
|              Rwanda|1996|           19.61|
|Falkland Islands ...|1997|            21.5|
|             Liberia|1998|           12.02|
|Falkland 

In [68]:
df_min_percent_change.orderBy('Year').show(31)

+--------------------+----+----------------+
|             Country|Year|MinPercentChange|
+--------------------+----+----------------+
|         Afghanistan|1981|           -9.11|
|         Afghanistan|1982|           -8.02|
| Antigua and Barbuda|1983|           -3.51|
| Antigua and Barbuda|1984|           -1.75|
|        Cook Islands|1985|           -1.41|
|Netherlands Antilles|1986|          -24.59|
|        Saint Helena|1987|           -21.3|
|          Mozambique|1988|           -2.88|
|             Somalia|1989|            -2.2|
|             Liberia|1990|          -12.82|
|              Kuwait|1991|          -55.45|
|             Somalia|1992|           -5.39|
|Bosnia and Herzeg...|1993|           -7.07|
|              Rwanda|1994|          -14.36|
|              Rwanda|1995|          -15.87|
|          Montserrat|1996|          -22.59|
|          Montserrat|1997|          -25.16|
|          Montserrat|1998|          -43.19|
|        Cook Islands|1999|           -2.99|
|        C

In [73]:
df_max_percent_change = df_max_percent_change.withColumnRenamed("Country", "Max Country")
df_max_percent_change = df_max_percent_change.withColumnRenamed("MaxPercentChange", "Max Percentage Change (%)")

In [74]:
df_min_percent_change = df_min_percent_change.withColumnRenamed("Country", "Min Country")
df_min_percent_change = df_min_percent_change.withColumnRenamed("MinPercentChange", "Min Percentage Change (%)")

In [75]:
df_combined = df_max_percent_change.join(
    df_min_percent_change.select('Min Country', 'Min Percentage Change (%)', 'Year'),
    on='Year'
)

### Answer to Question 3

In [76]:
df_combined.orderBy('Year').show(31,truncate=False)

+----+---------------------------------+-------------------------+----------------------+-------------------------+
|Year|Max Country                      |Max Percentage Change (%)|Min Country           |Min Percentage Change (%)|
+----+---------------------------------+-------------------------+----------------------+-------------------------+
|1981|Western Sahara                   |12.13                    |Afghanistan           |-9.11                    |
|1982|Western Sahara                   |11.12                    |Afghanistan           |-8.02                    |
|1983|French Guiana                    |14.29                    |Antigua and Barbuda   |-3.51                    |
|1984|Qatar                            |10.96                    |Antigua and Barbuda   |-1.75                    |
|1984|Saint Helena                     |10.96                    |Antigua and Barbuda   |-1.75                    |
|1985|French Guiana                    |12.5                     |Cook I

## Question 4

In [83]:
text_file = spark.read.text("romeo-juliet-pg1777.txt")

In [95]:
clean_text = text_file.select(lower(regexp_replace("value", "[^0-9a-zA-Z\\s]", "")).alias("text"))

In [96]:
clean_text.show(truncate=False)

+-----------------------------------------------------------------+
|text                                                             |
+-----------------------------------------------------------------+
|                                                                 |
|this etext file is presented by project gutenberg in             |
|cooperation with world library inc from their library of the     |
|future and shakespeare cdroms  project gutenberg often releases  |
|etexts that are not placed in the public domain                  |
|                                                                 |
|this etext has certain copyright implications you should read    |
|                                                                 |
|this electronic version of the complete works of william         |
|shakespeare is copyright 19901993 by world library inc and is    |
|provided by project gutenberg with permission  electronic and    |
|machine readable copies may be distributed so l

In [126]:
words = clean_text.select(explode(split("text", "\\s+")).alias("word"))

In [127]:
words = words.filter(words.word != '') #removing blanks

In [128]:
words.show()

+-----------+
|       word|
+-----------+
|       this|
|      etext|
|       file|
|         is|
|  presented|
|         by|
|    project|
|  gutenberg|
|         in|
|cooperation|
|       with|
|      world|
|    library|
|        inc|
|       from|
|      their|
|    library|
|         of|
|        the|
|     future|
+-----------+
only showing top 20 rows



In [129]:
word_counts = words.groupBy("word").agg(count("*").alias("count")).orderBy("count", ascending=False)

### Answer to Question 4

In [130]:
word_counts.show(truncate=False) #dataframe countains count of all words

+----+-----+
|word|count|
+----+-----+
|and |773  |
|the |748  |
|i   |585  |
|to  |580  |
|a   |483  |
|of  |474  |
|is  |383  |
|that|365  |
|my  |360  |
|in  |328  |
|you |323  |
|thou|277  |
|not |275  |
|with|268  |
|for |267  |
|me  |265  |
|this|258  |
|it  |237  |
|be  |234  |
|but |186  |
+----+-----+
only showing top 20 rows



In [132]:
word_counts.count()

4065

## Question 5

In [743]:
foreclosures = spark.read.json('durham-nc-foreclosure-2006-2016.json')

In [744]:
foreclosures.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           datasetid|              fields|            geometry|    record_timestamp|            recordid|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|foreclosure-2006-...|{217 E CORPORATIO...|{[-78.8922549, 36...|2017-03-06T12:41:...|629979c85b1cc68c1...|
|foreclosure-2006-...|{401 N QUEEN ST, ...|{[-78.895396, 35....|2017-03-06T12:41:...|e3cce8bbc3c9b804c...|
|foreclosure-2006-...|{403 N QUEEN ST, ...|{[-78.8950321, 35...|2017-03-06T12:41:...|311559ebfeffe7ebc...|
|foreclosure-2006-...|{918 GILBERT ST, ...|{[-78.8873774, 35...|2017-03-06T12:41:...|7ec0761bd385bab8a...|
|foreclosure-2006-...|{721 LIBERTY ST, ...|{[-78.888343, 35....|2017-03-06T12:41:...|c81ae2921ffca8125...|
|foreclosure-2006-...|{729 HOPKINS ST, ...|{[-78.888092, 35....|2017-03-06T12:41:...|ae17ea44c5918fd2d...|
|foreclosure-2006-...|{1302 E MAIN ST

In [745]:
restaurants = rdf.filter((rdf.Rpt_Area_Desc == 'Food Service') & (rdf.Status=='ACTIVE'))

In [746]:
restaurants_2 = restaurants.select("ID", "Premise_Name", "geolocation")
restaurants_2.show(5, truncate=False)

+-----+------------------------+-----------------------+
|ID   |Premise_Name            |geolocation            |
+-----+------------------------+-----------------------+
|56060|WEST 94TH ST PUB        |35.9207272, -78.9573299|
|58123|BROOKDALE DURHAM IFS    |36.0467802, -78.8895483|
|70266|SMOOTHIE KING           |35.9182655, -78.9593263|
|97837|HAMPTON INN & SUITES    |36.0183378, -78.9060312|
|58349|KROGER R 381 MEAT MARKET|35.9495321, -78.9211694|
+-----+------------------------+-----------------------+
only showing top 5 rows



In [747]:
ab = foreclosures.select("geometry").collect()
bd = restaurants_2.select("geolocation").collect()

In [748]:
type(ab[0][0].coordinates)

list

In [749]:
bd[0][0]

'35.9207272, -78.9573299'

In [750]:
foreclosures_2 = foreclosures.select(col("fields.geocode")[0].alias("lat1"), col("fields.geocode")[1].alias("lon1"))

In [751]:
foreclosures_2.show(truncate=False)

+----------+-----------+
|lat1      |lon1       |
+----------+-----------+
|36.0013755|-78.8922549|
|35.995797 |-78.895396 |
|35.995413 |-78.8950321|
|35.9957683|-78.8873774|
|35.993026 |-78.888343 |
|35.99217  |-78.888092 |
|35.9865799|-78.886681 |
|35.9815968|-78.8806365|
|35.992438 |-78.874621 |
|35.985903 |-78.869642 |
|35.97299  |-78.9041979|
|35.9666667|-78.9060606|
|35.965573 |-78.903483 |
|36.0367999|-78.8960511|
|35.972325 |-78.890249 |
|35.978108 |-78.889404 |
|35.976643 |-78.8692657|
|35.97492  |-78.868701 |
|35.979723 |-78.882454 |
|35.9803659|-78.8873413|
+----------+-----------+
only showing top 20 rows



In [752]:
restaurants_3 = restaurants_2.select(col("ID"), col("Premise_Name"), split(col("geolocation"), ",")[0].cast("double").alias("lat2"), 
            split(col("geolocation"), ",")[1].cast("double").alias("lon2"))

In [753]:
restaurants_3.show()

+------+--------------------+----------+-----------+
|    ID|        Premise_Name|      lat2|       lon2|
+------+--------------------+----------+-----------+
| 56060|    WEST 94TH ST PUB|35.9207272|-78.9573299|
| 58123|BROOKDALE DURHAM IFS|36.0467802|-78.8895483|
| 70266|       SMOOTHIE KING|35.9182655|-78.9593263|
| 97837|HAMPTON INN & SUITES|36.0183378|-78.9060312|
| 58349|KROGER R 381 MEAT...|35.9495321|-78.9211694|
| 58342|LEONE INTERNATION...|35.9852771| -78.898944|
| 57278|  PIZZA HUT DELIVERY|36.0586094|-78.9279599|
| 57190|       CHUBBYS TACOS|36.0094173|-78.9218729|
| 57585|              SUBWAY| 36.018111|-78.9102149|
|178137|DUKE UNIVERSITY W...| 36.001517|-78.9391734|
|181419|DICKEYS BARBECUE PIT|35.9550003|-78.9931015|
|179950|DUKE UNIVERSITY/T...|35.9950533|-78.9447645|
| 57048|                 KFC|36.0507859|-78.9044701|
|179953|DUKE UNIVERSITY/T...|35.9950533|-78.9447645|
|180462|TRIDENT PROVISION...|35.9953688|-78.9417564|
|188410|PURDUE PHARMA MAN...|36.1027227|-78.84

In [754]:
import math
from pyspark.sql.types import DoubleType
from haversine import haversine

def haversine_dist(lat1, lon1, lat2, lon2):
    
    print(lat1, lat2, lon1, lon2)
    
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    lon1 = math.radians(lon1)
    lon2 = math.radians(lon2)
    
    R = 3959  # radius of the earth in miles
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [755]:
from pyspark.sql.functions import udf, radians
distance_udf = udf(haversine_dist, DoubleType())

In [756]:
joined_df = foreclosures_2.crossJoin(restaurants_3)

In [757]:
joined_df.show()

+----------+-----------+------+--------------------+----------+-----------+
|      lat1|       lon1|    ID|        Premise_Name|      lat2|       lon2|
+----------+-----------+------+--------------------+----------+-----------+
|36.0013755|-78.8922549| 56060|    WEST 94TH ST PUB|35.9207272|-78.9573299|
|36.0013755|-78.8922549| 58123|BROOKDALE DURHAM IFS|36.0467802|-78.8895483|
|36.0013755|-78.8922549| 70266|       SMOOTHIE KING|35.9182655|-78.9593263|
|36.0013755|-78.8922549| 97837|HAMPTON INN & SUITES|36.0183378|-78.9060312|
|36.0013755|-78.8922549| 58349|KROGER R 381 MEAT...|35.9495321|-78.9211694|
|36.0013755|-78.8922549| 58342|LEONE INTERNATION...|35.9852771| -78.898944|
|36.0013755|-78.8922549| 57278|  PIZZA HUT DELIVERY|36.0586094|-78.9279599|
|36.0013755|-78.8922549| 57190|       CHUBBYS TACOS|36.0094173|-78.9218729|
|36.0013755|-78.8922549| 57585|              SUBWAY| 36.018111|-78.9102149|
|36.0013755|-78.8922549|178137|DUKE UNIVERSITY W...| 36.001517|-78.9391734|
|36.0013755|

In [758]:
joined_df = joined_df.dropna(subset=['lat1', 'lon1', 'lat2', 'lon2'])

In [759]:
joined_df.count()

2114838

In [787]:
result_df = joined_df.withColumn("distance", distance_udf(col("lat1"), 
                                                          col("lon1"), 
                                                          col("lat2"), 
                                                          col("lon2")))

In [788]:
result_df.show(truncate=False)

+----------+-----------+------+------------------------------------------+----------+-----------+------------------+
|lat1      |lon1       |ID    |Premise_Name                              |lat2      |lon2       |distance          |
+----------+-----------+------+------------------------------------------+----------+-----------+------------------+
|36.0013755|-78.8922549|56060 |WEST 94TH ST PUB                          |35.9207272|-78.9573299|6.655847169459578 |
|36.0013755|-78.8922549|58123 |BROOKDALE DURHAM IFS                      |36.0467802|-78.8895483|3.1409991184235353|
|36.0013755|-78.8922549|70266 |SMOOTHIE KING                             |35.9182655|-78.9593263|6.859349085002129 |
|36.0013755|-78.8922549|97837 |HAMPTON INN & SUITES                      |36.0183378|-78.9060312|1.4023667770503383|
|36.0013755|-78.8922549|58349 |KROGER R 381 MEAT MARKET                  |35.9495321|-78.9211694|3.930237197275593 |
|36.0013755|-78.8922549|58342 |LEONE INTERNATIONAL GROCERY/MEAT 

In [789]:
result_df = result_df.filter(col("distance")<=10.0)

In [790]:
result_df.show()

+----------+-----------+------+--------------------+----------+-----------+------------------+
|      lat1|       lon1|    ID|        Premise_Name|      lat2|       lon2|          distance|
+----------+-----------+------+--------------------+----------+-----------+------------------+
|36.0013755|-78.8922549| 56060|    WEST 94TH ST PUB|35.9207272|-78.9573299| 6.655847169459578|
|36.0013755|-78.8922549| 58123|BROOKDALE DURHAM IFS|36.0467802|-78.8895483|3.1409991184235353|
|36.0013755|-78.8922549| 70266|       SMOOTHIE KING|35.9182655|-78.9593263| 6.859349085002129|
|36.0013755|-78.8922549| 97837|HAMPTON INN & SUITES|36.0183378|-78.9060312|1.4023667770503383|
|36.0013755|-78.8922549| 58349|KROGER R 381 MEAT...|35.9495321|-78.9211694| 3.930237197275593|
|36.0013755|-78.8922549| 58342|LEONE INTERNATION...|35.9852771| -78.898944|1.1735382285674743|
|36.0013755|-78.8922549| 57278|  PIZZA HUT DELIVERY|36.0586094|-78.9279599|4.4295176792183755|
|36.0013755|-78.8922549| 57190|       CHUBBYS TACO

### Answer to Question 5

In [791]:
result_df = result_df.groupBy("ID", "Premise_Name").count()
result_df.show()

+------+--------------------+-----+
|    ID|        Premise_Name|count|
+------+--------------------+-----+
|152896|BLUE RIDGE DELI A...| 1827|
|106532|    BURGER KING 9653| 1691|
| 87764|DURHAM CONVENTION...| 1912|
|178383|SWEETIES SOUTHERN...| 1905|
| 56150|HOME PLATE RESTAU...| 1837|
| 98198|LA REGIA DOMINICA...| 1838|
| 58124|CARVER LIVING CEN...| 1881|
| 58126|ENO POINTE ASSIST...| 1745|
| 56102|TOP'S CHINA RESTA...| 1904|
| 55702|SHERATON IMPERIAL...| 1530|
| 56677|         CAPTAIN D S| 1718|
| 70783|RUTH'S CHRIS STEA...| 1654|
| 60441|   NOODLES & COMPANY| 1904|
| 56752|             Q SHACK| 1900|
| 58066|DURHAM SCHOOL OF ...| 1905|
| 55519| DEVINE'S RESTAURANT| 1909|
|174146|          BEYU CAFFE| 1912|
| 80343|HOMEWOOD SUITES R...| 1597|
| 58138|KINDRED HEALTH CA...| 1874|
|168435|ALOFT DURHAM DOWN...| 1912|
+------+--------------------+-----+
only showing top 20 rows



In [792]:
result_df = result_df.withColumnRenamed("count", "Foreclosures within 10 miles")

In [793]:
result_df.show(truncate=False)

+------+------------------------------------------+----------------------------+
|ID    |Premise_Name                              |Foreclosures within 10 miles|
+------+------------------------------------------+----------------------------+
|152896|BLUE RIDGE DELI AND CAFE                  |1827                        |
|106532|BURGER KING 9653                          |1691                        |
|87764 |DURHAM CONVENTION CENTER                  |1912                        |
|178383|SWEETIES SOUTHERN &VEGAN CATERING         |1905                        |
|56150 |HOME PLATE RESTAURANT                     |1837                        |
|98198 |LA REGIA DOMINICAN MEXICAN REST & GRILL 55|1838                        |
|58124 |CARVER LIVING CENTER IFS                  |1881                        |
|58126 |ENO POINTE ASSISTED LIVING IFS            |1745                        |
|56102 |TOP'S CHINA RESTAURANT                    |1904                        |
|55702 |SHERATON IMPERIAL HO